# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Big Data Real-Time Analytics com Python e Spark</font>

## <font color='blue'>Lab 5</font>

## <font color='blue'>Machine Learning com PySpark</font>

Leia os manuais em pdf no Capítulo 14 do curso com o material complementar.

![title](imagens/Lab5.png)

## <font color='blue'>Regressão Linear</font>

Usaremos Regressão Linear para prever o consumo de combustível de automóveis.

A variável **consumo** no dataset1.csv será a variável target (dependente) e as demais variáveis serão candidatas a features (variáveis preditoras ou independentes). 

Este é, portanto, será um problema de Regressão Linear Múltipla (quando temos mais de 1 variável preditora ou independente).

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.9.7


In [2]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
#!pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
#!pip install -q -U watermark

In [3]:
# Importa o findspark e inicializa
import findspark
findspark.init()

In [4]:
# Imports
import numpy as np
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

In [5]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

Author: Data Science Academy

numpy    : 1.20.3
pyspark  : 3.3.0
findspark: 2.0.1



## Carregando os Dados

In [6]:
# Criando o Spark Context
sc = SparkContext(appName = "Lab5")

22/08/24 17:50:21 WARN Utils: Your hostname, falcon.local resolves to a loopback address: 127.0.0.1; using 10.0.0.87 instead (on interface en0)
22/08/24 17:50:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/24 17:50:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
sc.setLogLevel("ERROR")

In [8]:
# Criando o Spark Session
spSession = SparkSession.builder.master("local").getOrCreate()

In [9]:
# Carregando os dados e gerando um RDD
dados = sc.textFile("dados/dataset1.csv")

In [10]:
type(dados)

pyspark.rdd.RDD

In [11]:
# Colocando o RDD em cache. Esse processo otimiza a performance
dados.cache()

dados/dataset1.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [12]:
# Número de registros
dados.count()

399

In [13]:
# Visualizando as primeiras linhas
dados.take(5)

['consumo,numero_cilindros,capacidade,horsepower,peso,aceleracao,ano,nome',
 '30,4,79,70,2074,19.5,71,peugeot 304',
 '30,4,88,76,2065,14.5,71,fiat 124b',
 '31,4,71,65,1773,19,71,toyota corolla 1200',
 '35,4,72,69,1613,18,71,datsun 1200']

In [14]:
# Removendo a primeira linha do arquivo (cabeçalho)
dados2 = dados.filter(lambda x: "horsepower" not in x)
dados2.count()

398

In [15]:
# Visualizando as primeiras linhas
dados2.take(5)

['30,4,79,70,2074,19.5,71,peugeot 304',
 '30,4,88,76,2065,14.5,71,fiat 124b',
 '31,4,71,65,1773,19,71,toyota corolla 1200',
 '35,4,72,69,1613,18,71,datsun 1200',
 '27,4,97,60,1834,19,71,volkswagen model 111']

## Limpeza dos Dados

Vamos verificar se há valores ausentes. RDDs são ótimos para processamento, mas ruins para exploração, então converteremos o RDD para DataFrame Spark e então para DataFrame Pandas.

In [16]:
# Converte RDD para DataFrame Spark
df_spark = dados2.map(lambda x: str(x)).map(lambda w: w.split(',')).toDF()

In [17]:
type(df_spark)

pyspark.sql.dataframe.DataFrame

In [18]:
# Converte DataFrame Spark para DataFrame Pandas
df_pandas = df_spark.toPandas()

In [19]:
type(df_pandas)

pandas.core.frame.DataFrame

In [20]:
df_pandas.head()

,_1,_2,_3,_4,_5,_6,_7,_8
0,30,4,79,70,2074,19.5,71,peugeot 304
1,30,4,88,76,2065,14.5,71,fiat 124b
2,31,4,71,65,1773,19,71,toyota corolla 1200
3,35,4,72,69,1613,18,71,datsun 1200
4,27,4,97,60,1834,19,71,volkswagen model 111


In [21]:
# Tem valores nulos?
df_pandas.isnull().values.any()

False

Não há valor nulo, mas será que há valor ausente (falta de informação)? Vamos checar.

In [22]:
# Verifica se há alguma linha com caracter especial "?"
total = np.sum(df_pandas.apply(lambda x: x.str.contains('\?')).values)
total

6

In [23]:
# Vejamos quais linhas e colunas têm o caracter especial
np.where(df_pandas.apply(lambda x: x.str.contains('\?')).values)

(array([ 48, 126, 330, 336, 354, 374]), array([3, 3, 3, 3, 3, 3]))

In [24]:
# Visualizando uma linha
df_pandas.iloc[330]

_1                    40.9
_2                       4
_3                      85
_4                       ?
_5                    1835
_6                    17.3
_7                      80
_8    renault lecar deluxe
Name: 330, dtype: object

In [25]:
# Usando um valor padrão para average HP (que será usado para preencher os valores ausentes)
mediaHP = sc.broadcast(75.0)

In [26]:
# Função para limpeza dos dados
def limpaDados(inputStr) :
    
    # Variável global
    global mediaHP
    
    # Lista de atributos
    attList = inputStr.split(",")
    
    # Substitui o caracter ? por um valor na coluna de índice 3
    hpValue = attList[3]
    if hpValue == "?":
        hpValue = mediaHP.value
       
    # Cria uma linha usando a função Row, limpando e convertendo os dados de string para float
    linhas = Row(consumo = float(attList[0]), 
                 numero_cilindros = float(attList[1]), 
                 capacidade = float(attList[2]), 
                 hosrsepower = float(hpValue), 
                 peso = float(attList[4]), 
                 aceleracao = float(attList[5]), 
                 ano = float(attList[6]), 
                 nome = attList[7]) 
    return linhas

In [27]:
# Executa a função no RDD
dados3 = dados2.map(limpaDados)
dados3.cache()
dados3.take(5)

[Row(consumo=30.0, numero_cilindros=4.0, capacidade=79.0, hosrsepower=70.0, peso=2074.0, aceleracao=19.5, ano=71.0, nome='peugeot 304'),
 Row(consumo=30.0, numero_cilindros=4.0, capacidade=88.0, hosrsepower=76.0, peso=2065.0, aceleracao=14.5, ano=71.0, nome='fiat 124b'),
 Row(consumo=31.0, numero_cilindros=4.0, capacidade=71.0, hosrsepower=65.0, peso=1773.0, aceleracao=19.0, ano=71.0, nome='toyota corolla 1200'),
 Row(consumo=35.0, numero_cilindros=4.0, capacidade=72.0, hosrsepower=69.0, peso=1613.0, aceleracao=18.0, ano=71.0, nome='datsun 1200'),
 Row(consumo=27.0, numero_cilindros=4.0, capacidade=97.0, hosrsepower=60.0, peso=1834.0, aceleracao=19.0, ano=71.0, nome='volkswagen model 111')]

## Análise Exploratória de Dados

In [28]:
# Cria um Dataframe
df_carros = spSession.createDataFrame(dados3)

In [29]:
# Estatísticas descritivas de duas variáveis (como exemplo)
df_carros.select("consumo", "numero_cilindros").describe().show()

+-------+-----------------+-----------------+
|summary|          consumo| numero_cilindros|
+-------+-----------------+-----------------+
|  count|              398|              398|
|   mean|23.51457286432161|5.454773869346734|
| stddev|7.815984312565782|1.701004244533212|
|    min|              9.0|              3.0|
|    max|             46.6|              8.0|
+-------+-----------------+-----------------+



In [30]:
# Encontrando a correlação entre a variável target com as variáveis preditoras (exceto o nome)
for i in df_carros.columns:
    if not(isinstance(df_carros.select(i).take(1)[0][0], str)) :
        print("Correlação da Variável Target com:", i, df_carros.stat.corr('consumo', i))

Correlação da Variável Target com: consumo 1.0
Correlação da Variável Target com: numero_cilindros -0.7753962854205546
Correlação da Variável Target com: capacidade -0.8042028248058979
Correlação da Variável Target com: hosrsepower -0.7747041523498721
Correlação da Variável Target com: peso -0.8317409332443347
Correlação da Variável Target com: aceleracao 0.42028891210164976
Correlação da Variável Target com: ano 0.5792671330833098


## Pré-Processamento dos Dados

Leia o manual em pdf no Capítulo 14 sobre a diferença entre Vetor Denso e Vetor Esparso.

In [31]:
# Convertendo para um LabeledPoint (target, Vector[features])
# Remove colunas não relevantes ou com baixa correlação
def transformaVar(row) :
    obj = (row["consumo"], Vectors.dense([row["peso"], row["capacidade"], row["numero_cilindros"]]))
    return obj

In [32]:
# Aplica a função no RDD e cria outro RDD
dados4 = dados3.map(transformaVar)

In [33]:
# Visualiza
dados4.take(5)

[(30.0, DenseVector([2074.0, 79.0, 4.0])),
 (30.0, DenseVector([2065.0, 88.0, 4.0])),
 (31.0, DenseVector([1773.0, 71.0, 4.0])),
 (35.0, DenseVector([1613.0, 72.0, 4.0])),
 (27.0, DenseVector([1834.0, 97.0, 4.0]))]

In [34]:
# Converte o RDD para DataFrame do Spark
df_carros = spSession.createDataFrame(dados4, ["label", "features"])

In [35]:
# Visualiza label (y) e atributos (x)
df_carros.select("label","features").show(10)

+-----+------------------+
|label|          features|
+-----+------------------+
| 30.0| [2074.0,79.0,4.0]|
| 30.0| [2065.0,88.0,4.0]|
| 31.0| [1773.0,71.0,4.0]|
| 35.0| [1613.0,72.0,4.0]|
| 27.0| [1834.0,97.0,4.0]|
| 26.0| [1955.0,91.0,4.0]|
| 24.0|[2278.0,113.0,4.0]|
| 25.0| [2126.0,97.5,4.0]|
| 23.0| [2254.0,97.0,4.0]|
| 20.0|[2408.0,140.0,4.0]|
+-----+------------------+
only showing top 10 rows



In [36]:
# Divisão em dados de Treino e de Teste com split 70/30
(dados_treino, dados_teste) = df_carros.randomSplit([0.7, 0.3])

In [37]:
dados_treino.count()

275

In [38]:
dados_teste.count()

123

## Machine Learning

In [39]:
# Cria o objeto
linearReg = LinearRegression()

In [40]:
# Treina o objeto com dados e cria o modelo
modelo = linearReg.fit(dados_treino)

In [41]:
print(modelo)

LinearRegressionModel: uid=LinearRegression_6aa41a0c4c72, numFeatures=3


In [42]:
# Imprimindo os coeficientes (o que o modelo aprendeu)
print("Coeficientes: " + str(modelo.coefficients))
print("Intercepto: " + str(modelo.intercept))

Coeficientes: [-0.005929028015916703,-0.014540341062653154,-0.2348848556095086]
Intercepto: 45.33388250950223


In [43]:
# Previsões com dados de teste
predictions = modelo.transform(dados_teste)

In [44]:
# Visualiza as previsões
predictions.select("features", "prediction").show()

+------------------+------------------+
|          features|        prediction|
+------------------+------------------+
|[4376.0,307.0,8.0]|13.045492360740155|
|[4615.0,360.0,8.0]|10.857816588615442|
|[4633.0,429.0,8.0]|  9.74781055100587|
|[4499.0,350.0,8.0]|11.690987249088316|
|[4906.0,400.0,8.0]| 8.550855793477552|
|[4951.0,455.0,8.0]|7.4843307743153815|
|[3988.0,350.0,8.0]|14.720720565221747|
|[4274.0,350.0,8.0]|13.025018552669572|
|[4464.0,400.0,8.0]|11.171486176512737|
|[4502.0,350.0,8.0]|11.673200165040562|
|[4654.0,360.0,8.0]|10.626584495994692|
|[4735.0,440.0,8.0]| 8.983105941693182|
|[5140.0,400.0,8.0]| 7.163463237753049|
|[3086.0,455.0,8.0]| 18.54196802400003|
|[4096.0,318.0,8.0]|14.545676453507642|
|[4209.0,350.0,8.0]|13.410405373704155|
|[4237.0,318.0,8.0]|13.709683503263388|
|[4354.0,454.0,8.0]|11.038500840880303|
|[4215.0,351.0,8.0]|13.360290864546002|
|[3158.0,250.0,6.0]|21.565617635916944|
+------------------+------------------+
only showing top 20 rows



In [45]:
# Coeficiente de determinação R2
avaliador = RegressionEvaluator(predictionCol = "prediction", labelCol = "label", metricName = "r2")

In [46]:
# Resultado
avaliador.evaluate(predictions) 

0.6964971456363918

# Fim